# ROB2004 Final Project - pick and place objects

The goal of this project is to solve a simple manipulation task: picking up objects and moving them in a bowl.

## Instructions
* Answer all questions in the notebook
* You will need to submit on Brightspace: 
    1. the code you wrote to answer the questions in a Jupyter Notebook. The code should be runnable as is.
    2. a 2-3 pages report in pdf format (pdf only) detailing the methodology you followed to answer the questions as well as answers to the questions that require a typeset answer. You may add the plots in the report (does not count for the page limit) or in the Jupyter notebook.
* This is an individual project

We will use a model of the Frank-Emika Panda robot. This robot has 7 revolute joints and its kinematics is described in the picture below.

You will need to install pybullet on your computer. Please reach out as soon as possible for help if you do not know how to do it.

<div>
<img src="./panda.jpg" width="200"/>
</div>

# Question

Implement a controller using the code skeletong below in order to get the robot to go and pick up the red blocks and drop them in the green bowl. The bowl position is $(-0.3,0.55,0.65)$ (in spatial frame coordinates) and the blocks positions are $(0.35,0.58,0.65)$ and $(0.15,0.67,0.65)$ respectively.

<div>
<img src="./manipulation_scene.jpg" width="300"/>
</div>

You are free to use the methods that you want to solve the task, with the following constraints:
* You cannot use any external library apart from numpy and scipy
* You need to use at least one controller in the end-effector space
* You need to generate smooth motions
* You need to compensate for the gravity of the robot

In your report, describe the controller and justify your choices. Analyze the behavior of the system (include plots of the end-effector trajectories, velocities, joint trajectories, etc as you see fit).

In [1]:
import numpy as np
import time

import matplotlib.pyplot as plt

from panda import Simulator, PandaRobot

np.set_printoptions(precision=4, suppress = True)

pybullet build time: Aug 21 2022 18:46:39


# Helper functions

We provide a set of helper functions (forward kinematics, Jacobians, gravity terms) that can be used to implement the desired controllers. These functions are implemented in the PandaRobot class (in the panda.py file).
Examples are shown below:

In [2]:
# create an instance of the helper class
my_robot = PandaRobot()

# compute the forward kinematics for an arbitrary joint configuration
q = np.random.uniform(-1,1,7)
pose = my_robot.FK(np.array([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,np.pi/4]))
# pose = my_robot.FK(q)
print(f'the pose of the end-effector for joint configuration\n\n {q}\n\n is\n\n {pose}')

the pose of the end-effector for joint configuration

 [ 0.327  -0.439  -0.1746  0.0077  0.71   -0.9357 -0.7461]

 is

 [[ 0.      1.      0.      0.    ]
 [ 1.     -0.     -0.      0.5545]
 [ 0.      0.     -1.      0.6245]
 [ 0.      0.      0.      1.    ]]


In [3]:
# we can also get the Jacobian in various frames 
# it is sufficient to pass the name of the frame as an argument as shown below

q = np.random.uniform(-1,1,7)

spatial_jacobian = my_robot.get_jacobian(q, 'S')
body_jacobian = my_robot.get_jacobian(q, 'B')

# we can even get the Jacobian in a frame place at the same position as the end-effector 
# frame but oriented like the spatial frame
oriented_jacobian = my_robot.get_jacobian(q, 'O')

print(f'for joint configuration \n\n {q} \n \n the spatial Jacobian is \n\n {spatial_jacobian}\n\n')
print(f'the body Jacobian is \n\n {body_jacobian}\n\n')
print(f'and the Jacobian in a frame placed at the same position as the end-effector but oriented like the spatial frame is\n\n {oriented_jacobian}')

for joint configuration 

 [ 0.8502 -0.5685  0.214   0.3007  0.6813  0.0103 -0.4231] 
 
 the spatial Jacobian is 

 [[ 0.     -0.6598  0.4046  0.5103  0.6112  0.0153 -0.6193]
 [ 0.     -0.7514 -0.3552  0.8524 -0.453   0.8266  0.4495]
 [ 1.      0.      0.8427  0.1143  0.649   0.5625 -0.6438]
 [ 0.      0.2502  0.1183 -0.557   0.1967 -0.8413 -0.1967]
 [ 0.     -0.2197  0.1347  0.3205  0.2801 -0.1841 -0.3615]
 [ 0.      0.     -0.      0.0967  0.0103  0.2934 -0.0632]]


the body Jacobian is 

 [[-0.3421  0.864  -0.0894 -0.9495  0.0036 -0.9351  0.    ]
 [ 0.6845  0.4984  0.2904 -0.3136  0.0096  0.3545  0.    ]
 [-0.6438  0.0708 -0.9527 -0.0065 -0.9999  0.      1.    ]
 [-0.267  -0.2883  0.005   0.0871  0.0833  0.0379  0.    ]
 [-0.1929  0.4787 -0.0547 -0.264  -0.0316  0.1001  0.    ]
 [-0.0632  0.1476 -0.0171  0.0111 -0.     -0.088  -0.    ]]


and the Jacobian in a frame placed at the same position as the end-effector but oriented like the spatial frame is

 [[ 0.     -0.6598  0.4046  0.

In [4]:
# finally it is possible to compute the effect of gravity on the joints
# this can for example be used to do gravity compensation when controlling the robot

q = np.random.uniform(-1,1,7)

g = my_robot.g(q)

print(f'for joint configuration \n\n {q} \n \n the gravity force seen on the joints is \n \n {g}')

for joint configuration 

 [-0.3955 -0.7603  0.535  -0.8721 -0.797   0.1966 -0.8511] 
 
 the gravity force seen on the joints is 
 
 [ 0.     21.8226 -7.0633  0.0616 -0.4052  1.0747  0.    ]


# Code Skeleton for the simulation

Feel free to change the `run_time` variable to match your needs. The class `Simulator` contains all the code for the simulation. The code skeleton currently implement a simple PD controller in joint space that just keeps a desired current position.

Importantly you can control both the joints and the gripper of the robot:
* `simulator.send_joint_torque(joint_torques)` sends a vector of dimension 7 setting the torques of the joints
* `simulator.gripper_move(gripper_position)` sends a vector of dimension 2 setting the position of the fingers of the gripper. The fingers can have positions from 0 to 0.04cm (you can fully close the gripper by setting `simulator.gripper_move([0,0])` and fully open it by setting `simulator.gripper_move([0.04,0.04)`)

In [5]:
# here we create a simulation and reset the state of the robot
simulator = Simulator()
simulator.reset_state([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,np.pi/4])

# we create a robot object so we can use its helper functions
my_robot = PandaRobot()

# duration of the simulation
run_time = 60.

# simulation time step
dt = 0.005
num_steps = int(run_time/dt)


# we store information
ndofs = 7 # number of degrees of freedom (excluding the gripper)
measured_joint_pos = np.zeros([num_steps,ndofs])
measured_joint_vel = np.zeros_like(measured_joint_pos)
desired_torques = np.zeros_like(measured_joint_pos)
desired_joint_pos = np.zeros_like(measured_joint_pos)
desired_joint_vel = np.zeros_like(measured_joint_pos)
time = np.zeros([num_steps])

'''Record EE Coord'''
measured_ee_pos = np.zeros([num_steps,3])
measured_ee_vel = np.zeros_like(measured_ee_pos)

### this is the main control loop
for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    
    # we get the position and velocities of the joints
    q, dq = simulator.get_state()
    measured_joint_pos[i,:] = q
    measured_joint_vel[i,:] = dq
    pose = my_robot.FK(q)
    measured_ee_pos[i,:] = pose[0:3,3]
    if i > 0: # Get rid of first iter
        measured_ee_vel[i,:] = (measured_ee_pos[i,:] - measured_ee_pos[i-1,:])/dt
    
    ## controller: TODO HERE IMPLEMENT YOUR CONTROLLER TO SOLVE THE TASK
    ## you will need to replace the naive PD controller implemented below

    origin = np.array([0.35,0.58,0.45])
    block1 = np.array([0.35,0.58,0.65])
    block2 = np.array([0.15,0.67,0.65])
    # desired joint positions and velocities
    q_des = np.array([0.,0,0.0,-np.pi/2.,0.,(q[1] + q[3]),q[0]+np.pi/4])
#     q_des = np.array([0.,0,0.0,-np.pi/2.,0.,np.pi/2.,0])
    dq_des = np.zeros([7])
    
    # we save the desired positions/velocities for later plotting 
    desired_joint_pos[i,:] = q_des
    desired_joint_vel[i,:] = dq_des
    
    ##PD controller
    # the PD gains
    P = np.array([100., 100., 100., 100., 100., 100., 50.])
    D = np.array([10.,10,10,10,10,10,1.])

    error = q_des - q # the position error for all the joints
    d_error = dq_des-dq # the velocity error for all the joints
    
    K = np.array([50,50,50])
    B = np.array([5,5,5])
    jacob = my_robot.get_jacobian(q, 'O')[3:,:]
    # we compute the desired torques as a PD controller
#     joint_torques = np.diag(P) @ error + np.diag(D) @ d_error + my_robot.g(q)
    
    joint_torques = jacob.T @ (np.diag(K) @ (origin - measured_ee_pos[i,:]) - np.diag(B) @ (measured_ee_vel[i,:])) + my_robot.g(q)
    joint_torques[2] = 100 * error[2] + 10* d_error[2]
    joint_torques[4] = 100 * error[4] + 10* d_error[4]
    joint_torques[5] = 100 * error[5] + 10* d_error[5]
    joint_torques[6] = 100 * error[6] + 10* d_error[6]
# joint 5 must be vertical
    qdes_5 = q[1] + q[3]
# joint 6 must be parallel to edge
#     joint_torques[6] = -joint_torques[0]
#     joint_torques += my_robot.g(q)
    if i % 100 == 0:
        print(qdes_5)
    
    # we start by closing the gripper and then open it after 5 seconds
    gripper_position = [0,0]
    if time[i] > 5.0:
        gripper_position = [0.04, 0.04]
    
    # we send them to the robot and do one simulation step
    simulator.send_joint_torque(joint_torques)
    simulator.gripper_move(gripper_position)
    simulator.step()

-1.5707963267948966
-0.759974293499807
-0.17003730764103286
0.24230966316198443
0.43553605019795494
0.6290875756428662
0.8835531901142328
1.0823507768411906
1.175861545361736
1.1905361459423134
1.1237425912002907
1.0638971940649369
1.052461205860482
1.0677508111041514
1.0770049473304972
1.0676830415949345
1.050925988635752
1.0445121136190505
1.0544495845044746
1.0719911585694009
1.0859474661941753
1.0938350196183915
1.1020556881066146
1.103845841248529
1.1062188790220044
1.1150266874181394
1.119964356402018
1.1172751755353283
1.1115842333581232
1.1091979049329173
1.1115441975213183
1.1150947930127633
1.116100289495615
1.114256602641131
1.112107989989784
1.111812638343973
1.1131637022362961
1.114417083804822
1.1143861722722863
1.113441572733828
1.1127397334377092
1.11289517249546
1.1135300156016938
1.1139028475919992
1.1137223367104943
1.1133101175639997
1.1131264983909166
1.1132883133946523
1.1135474955051996
1.113627423844425
1.1134981814123561
1.1133405034677097
1.1133144586091812
1.